In [1]:
from glob import glob
import numpy as np

In [2]:
from rdkit import Chem

In [3]:
def read_sdf(sdf):
    with open(sdf, "r") as f:
        txt = f.read().rstrip()
    return txt

In [4]:
def get_ncharges_coords(sdf):
    mol = Chem.MolFromMolBlock(sdf)
   #mol = Chem.AddHs(mol)
    # rdkit molobj
    ncharges = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    conf = mol.GetConformer()
    coords = np.asarray(conf.GetPositions())
    return ncharges, coords

In [5]:
def cutoff_func(R_ij, central_cutoff=4.8, central_decay=1):
    if R_ij <= (central_cutoff - central_decay):
        func = 1.
    elif ((central_cutoff - central_decay) < R_ij) and (R_ij <= (central_cutoff + central_decay)):
        func = 0.5 * (1. + np.cos((np.pi * R_ij - central_cutoff + central_decay)/central_decay))
    else:
        func = 0.
    return func

In [6]:
def get_atomic_CM(ncharges, coords, max_natoms, central_cutoff=4.8, central_decay=1):
    size = int((max_natoms + 1)*max_natoms / 2)
    rep = np.zeros((len(ncharges), size))
    
    # central atom loop
    for k in range(len(ncharges)):
        M = np.zeros((len(ncharges), len(ncharges)))
        for i in range(len(ncharges)):
            R_ik = np.linalg.norm(coords[i]-coords[k])
           # print('R_ik', R_ik)
            f_ik = cutoff_func(R_ik, central_cutoff=central_cutoff,
                              central_decay=central_decay)
            for j in range(len(ncharges)):
                if i <=j:
                    if i == j:
                        M[i,j] = 0.5 * ncharges[i]**2.4 * f_ik**2
                        M[j,i] = M[i,j]

                    else:
                        R_jk = np.linalg.norm(coords[j]-coords[k])
                      #  print('R_jk', R_jk)
                        f_jk = cutoff_func(R_jk, central_cutoff=central_cutoff,
                                          central_decay=central_decay)
                        R_ij = np.linalg.norm(coords[i]-coords[j])
                      #  print('R_ij', R_ij)
                        f_ij = cutoff_func(R_ij, central_cutoff=central_cutoff,
                                          central_decay=central_decay)
                        M[i,j] = (ncharges[i]*ncharges[j]/R_ij)*f_ik*f_jk*f_ij
                        M[j,i] = M[i,j]


        # concat upper triangular and diagonal
        upper_triang = M[np.triu_indices(len(M))]
        s_upper_triang = np.sort(upper_triang)[::-1]
        
        # pad to full size
        n_zeros = size - len(s_upper_triang)
        zeros = np.zeros(n_zeros)
        rep[k] = np.concatenate((s_upper_triang, zeros))

    return rep

In [7]:
target_sdfs = sorted(glob("targets/*.sdf"))
target_sdfs

['targets/qm9.sdf', 'targets/vitc.sdf', 'targets/vitd.sdf']

In [8]:
qm9_amons_files = sorted(glob("amons-qm9/*.sdf"))

In [9]:
qm9_amons_sdfs = [read_sdf(x) for x in qm9_amons_files]

In [10]:
conf_data = [get_ncharges_coords(x) for x in qm9_amons_sdfs]

In [11]:
ncharges_list, coords_list = zip(*conf_data)

In [12]:
qm9_ncharges = ncharges_list

In [13]:
qm9_ncharges

([8],
 [6, 6],
 [6, 7],
 [8, 6],
 [6, 8, 8],
 [6, 8, 7],
 [6, 6, 7, 7],
 [8, 6, 6, 7],
 [8, 6, 6, 6],
 [6, 7, 6, 8],
 [6, 7, 6, 8, 8],
 [6, 8, 8, 7, 6],
 [8, 6, 6, 7, 7, 6],
 [8, 6, 6, 7, 6, 8],
 [8, 6, 6, 7, 6, 8, 8],
 [6, 7, 6, 8, 8, 7, 6])

In [14]:
qm9_reps = [np.array(get_atomic_CM(np.array(ncharges_list[i]),
                                                                        np.array(coords_list[i]), 
                                                                        max_natoms=28))
            for i in range(len(ncharges_list))]

In [15]:
qm9_reps = np.array(qm9_reps)

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [16]:
qm9_reps[0].shape

(1, 406)

In [17]:
qm9_reps[-1]

array([[73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 53.3587074 , 53.3587074 , ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
qm9_amons_labels = [t.split("/")[-1].split(".sdf")[0] for t in qm9_amons_files]

In [19]:
vitc_amons_files = sorted(glob("amons-vitc/*.sdf"))

In [20]:
vitc_amons_sdfs = [read_sdf(x) for x in vitc_amons_files]

In [21]:
conf_data = [get_ncharges_coords(x) for x in vitc_amons_sdfs]

In [22]:
ncharges_list, coords_list = zip(*conf_data)

In [23]:
vitc_ncharges = ncharges_list

In [24]:
vitc_reps = [np.array(get_atomic_CM(np.array(ncharges_list[i]), np.array(coords_list[i]), 
                                                         max_natoms=28)) for i in 
            range(len(ncharges_list))]

In [25]:
vitc_reps = np.array(vitc_reps)

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [26]:
vitc_amons_labels = [t.split("/")[-1].split(".sdf")[0] for t in vitc_amons_files]

In [27]:
vitd_amons_files = sorted(glob("amons-vitd/*.sdf"))

In [28]:
vitd_amons_sdfs = [read_sdf(x) for x in vitd_amons_files]

In [29]:
conf_data = [get_ncharges_coords(x) for x in vitd_amons_sdfs]

In [30]:
ncharges_list, coords_list = zip(*conf_data)

In [31]:
vitd_ncharges = ncharges_list

In [32]:
vitd_reps = [np.array(get_atomic_CM(np.array(ncharges_list[i]), np.array(coords_list[i]),
                                                                         max_natoms=28))
    for i in range(len(ncharges_list))]

In [33]:
vitd_reps = np.array(vitd_reps)

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [34]:
vitd_amons_labels = [t.split("/")[-1].split(".sdf")[0] for t in vitd_amons_files]

In [35]:
# np save 

In [36]:
np.savez("amons_aCM_data.npz", 
         vitd_amons_labels=vitd_amons_labels,
         vitc_amons_labels=vitc_amons_labels,
         qm9_amons_labels=qm9_amons_labels,
         vitd_amons_ncharges=vitd_ncharges,
         vitc_amons_ncharges=vitc_ncharges,
         qm9_amons_ncharges=qm9_ncharges,
         vitd_amons_reps=vitd_reps,
         vitc_amons_reps=vitc_reps,
         qm9_amons_reps=qm9_reps)

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [37]:
vitd_reps[0].shape

(1, 406)